# Get BLIP and Siamese model embeddings for all images
These embeddings will serve as our database to recommend furniture to users

## Merge test, training and validation into one dataframe

In [23]:
import pandas as pd

In [24]:
test_df = pd.read_csv('testing_with_blip.csv')
training_df= pd.read_csv('training_with_blip.csv')
validation_df = pd.read_csv('validation_with_blip.csv')

In [25]:
test_df.shape, training_df.shape, validation_df.shape

((228, 4), (1819, 4), (227, 4))

In [26]:
test_df = test_df.drop(test_df.columns[0], axis=1)
training_df = training_df.drop(training_df.columns[0], axis=1)
validation_df = validation_df.drop(validation_df.columns[0], axis=1)

In [27]:
display(test_df.head())
display(training_df.head())
display(validation_df.head())


,group,image_path,blip
0,453,sofas/sofa-beds/91503_image_0_resized.jpg,"[0.38693681359291077, 0.19435164332389832, -0...."
1,104,sofas/4-seaters-up/94024_image_0_resized.jpg,"[0.6286948919296265, -0.5614968538284302, -0.1..."
2,185,sofas/l-shape/91717_image_0_resized.jpg,"[0.42798036336898804, 0.20924703776836395, 0.0..."
3,115,sofas/4-seaters-up/88163_image_0_resized.jpg,"[0.021315090358257294, 0.19990620017051697, -0..."
4,811,chairs/office-chairs/25435_image_0_resized.jpg,"[0.2223481684923172, 0.12761016190052032, -0.0..."


,group,image_path,blip
0,13,sofas/2-seaters/91513_image_0_resized.jpg,"[-0.28666922450065613, -0.6765906810760498, -0..."
1,13,sofas/2-seaters/87437_image_0_resized.jpg,"[0.20910055935382843, -0.016624245792627335, 0..."
2,14,sofas/2-seaters/88716_image_0_resized.jpg,"[-0.27765217423439026, 0.284512460231781, -0.0..."
3,14,sofas/2-seaters/94018_image_0_resized.jpg,"[-0.5567976236343384, 0.18054495751857758, -0...."
4,15,sofas/2-seaters/91579_image_0_resized.jpg,"[0.01144308689981699, -0.23772938549518585, -0..."


,group,image_path,blip
0,391,sofas/outdoor-sofas/14619_image_0_resized.jpg,"[0.05484333261847496, -0.4026709496974945, 0.1..."
1,93,sofas/3-seaters/46964_image_0_resized.jpg,"[-0.2954059839248657, -0.08965714275836945, -0..."
2,61,sofas/3-seaters/92228_image_0_resized.jpg,"[0.3323415517807007, -0.3226076364517212, -0.1..."
3,332,sofas/lounge-chairs/49322_image_0_resized.jpg,"[-0.1696816384792328, -0.2943868339061737, 0.0..."
4,879,chairs/stools-ottomans/41477_image_0_resized.jpg,"[-0.04996024817228317, 0.190990149974823, -0.0..."


In [28]:
# merge all dataframes
all_df = pd.concat([test_df, training_df, validation_df], ignore_index=True)
all_df.shape

(2274, 3)

In [29]:
all_df

,group,image_path,blip
0,453,sofas/sofa-beds/91503_image_0_resized.jpg,"[0.38693681359291077, 0.19435164332389832, -0...."
1,104,sofas/4-seaters-up/94024_image_0_resized.jpg,"[0.6286948919296265, -0.5614968538284302, -0.1..."
2,185,sofas/l-shape/91717_image_0_resized.jpg,"[0.42798036336898804, 0.20924703776836395, 0.0..."
3,115,sofas/4-seaters-up/88163_image_0_resized.jpg,"[0.021315090358257294, 0.19990620017051697, -0..."
4,811,chairs/office-chairs/25435_image_0_resized.jpg,"[0.2223481684923172, 0.12761016190052032, -0.0..."
...,...,...,...
2269,331,sofas/lounge-chairs/89623_image_0_resized.jpg,"[0.1230996698141098, -0.3463379740715027, -0.4..."
2270,87,sofas/3-seaters/91602_image_0_resized.jpg,"[0.43745896220207214, -0.024218738079071045, 0..."
2271,316,sofas/leather-sofas/88717_image_0_resized.jpg,"[-0.11025682836771011, -0.4462471008300781, -0..."
2272,856,chairs/office-chairs/79309_image_0_resized.jpg,"[-0.0018005091696977615, -0.367832213640213, 0..."


In [30]:
# see if any duplicate image_path
all_df[all_df.duplicated(subset='image_path')]
all_df = all_df.drop_duplicates(subset='image_path')
all_df.shape

(2274, 3)

In [31]:
all_df.to_csv('all_with_blip.csv')

## Get the embeddings of all images from our Siamese model

In [ ]:
import ast
import pandas as pd

import torch
import torch.nn as nn

### 1. Define Siamese Network

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, input_size, embedding_size=128):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(256, embedding_size),
        )
    
    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights_path = '/kaggle/input/siamese-triplets/pytorch/final-training/6/5_epoch_3linear_dropout_lr_5.pt'
siamese_model = SiameseNetwork(input_size=768, embedding_size=128).to(device)
siamese_model.load_state_dict(torch.load(weights_path, map_location=device))
siamese_model.eval()

SiameseNetwork(
  (fc): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
  )
)

### 2. Load dataset

In [ ]:
df_dtype = {
    'image_path': 'str',
    'blip': 'object'
}

In [ ]:
df_all_blip = pd.read_csv('/kaggle/input/all-images/all_with_blip.csv', dtype=df_dtype)
df_all_blip['blip_siamese'] = pd.Series(dtype='object')
df_all_blip.set_index('image_path')

,Unnamed: 0,group,blip,blip_siamese
image_path,,,,
sofas/sofa-beds/91503_image_0_resized.jpg,0,453,"[0.38693681359291077, 0.19435164332389832, -0....",NaN
sofas/4-seaters-up/94024_image_0_resized.jpg,1,104,"[0.6286948919296265, -0.5614968538284302, -0.1...",NaN
sofas/l-shape/91717_image_0_resized.jpg,2,185,"[0.42798036336898804, 0.20924703776836395, 0.0...",NaN
sofas/4-seaters-up/88163_image_0_resized.jpg,3,115,"[0.021315090358257294, 0.19990620017051697, -0...",NaN
chairs/office-chairs/25435_image_0_resized.jpg,4,811,"[0.2223481684923172, 0.12761016190052032, -0.0...",NaN
...,...,...,...,...
sofas/lounge-chairs/89623_image_0_resized.jpg,2269,331,"[0.1230996698141098, -0.3463379740715027, -0.4...",NaN
sofas/3-seaters/91602_image_0_resized.jpg,2270,87,"[0.43745896220207214, -0.024218738079071045, 0...",NaN
sofas/leather-sofas/88717_image_0_resized.jpg,2271,316,"[-0.11025682836771011, -0.4462471008300781, -0...",NaN


In [ ]:
for image_path, row in df_all_blip.iterrows():
    blip_embedding = ast.literal_eval(row['blip'])
    features = torch.tensor(blip_embedding).float()

    # siamese model
    with torch.no_grad():
        embedding_siamese = siamese_model(features.to(device).unsqueeze(0))
    validation_embedding = embedding_siamese.cpu().squeeze(0)
    df_all_blip.at[image_path, 'blip_siamese'] = validation_embedding.numpy().tolist()

In [ ]:
df_all_blip.head()

,Unnamed: 0,group,image_path,blip,blip_siamese
0,0,453,sofas/sofa-beds/91503_image_0_resized.jpg,"[0.38693681359291077, 0.19435164332389832, -0....","[0.01813609153032303, -0.14332135021686554, -0..."
1,1,104,sofas/4-seaters-up/94024_image_0_resized.jpg,"[0.6286948919296265, -0.5614968538284302, -0.1...","[0.22950240969657898, -0.12034033983945847, -0..."
2,2,185,sofas/l-shape/91717_image_0_resized.jpg,"[0.42798036336898804, 0.20924703776836395, 0.0...","[0.09146980941295624, -0.23594890534877777, -0..."
3,3,115,sofas/4-seaters-up/88163_image_0_resized.jpg,"[0.021315090358257294, 0.19990620017051697, -0...","[0.23008421063423157, -0.1584370881319046, -0...."
4,4,811,chairs/office-chairs/25435_image_0_resized.jpg,"[0.2223481684923172, 0.12761016190052032, -0.0...","[0.19579604268074036, -0.28381842374801636, 0...."


In [ ]:
df_all_blip = df_all_blip.drop(df_all_blip.columns[0], axis=1)

In [ ]:
df_all_blip.head()

,group,image_path,blip,blip_siamese
0,453,sofas/sofa-beds/91503_image_0_resized.jpg,"[0.38693681359291077, 0.19435164332389832, -0....","[0.01813609153032303, -0.14332135021686554, -0..."
1,104,sofas/4-seaters-up/94024_image_0_resized.jpg,"[0.6286948919296265, -0.5614968538284302, -0.1...","[0.22950240969657898, -0.12034033983945847, -0..."
2,185,sofas/l-shape/91717_image_0_resized.jpg,"[0.42798036336898804, 0.20924703776836395, 0.0...","[0.09146980941295624, -0.23594890534877777, -0..."
3,115,sofas/4-seaters-up/88163_image_0_resized.jpg,"[0.021315090358257294, 0.19990620017051697, -0...","[0.23008421063423157, -0.1584370881319046, -0...."
4,811,chairs/office-chairs/25435_image_0_resized.jpg,"[0.2223481684923172, 0.12761016190052032, -0.0...","[0.19579604268074036, -0.28381842374801636, 0...."


In [ ]:
df_all_blip.to_csv('/kaggle/working/all_blip_siamese.csv')